# An Example 2D optimisation
## Inspired by the following notebook:
[scikit-optimize benchmarks](https://github.com/scikit-optimize/scikit-optimize.github.io/blob/master/notebooks/strategy-comparison.md)

The [Branin-Hoo function](https://www.sfu.ca/~ssurjano/branin.html) is a commonly used function for benchmarking black-box global optimisation algorithms

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import sklearn.gaussian_process as sk_gp
import seaborn as sns; sns.set() # prettify matplotlib

In [ ]:
import turbo as tb
import turbo.gui.jupyter as tg
import turbo.modules as tm
import turbo.plotting as tp

In [ ]:
# Make deterministic
np.random.seed(42)

In [ ]:
class BraninHoo:
    def __init__(self):
        self.bounds = [('x', -5., 10.), ('y', 0., 15.)]

        self.global_optima = [(-pi, 12.275), (pi, 2.275), (9.42478, 2.475)]
        self.optimal_y = 0.39788735772973816
        self.max_y = self(-5, 0)

        self.vectorized = np.vectorize(self.__call__)

    def __call__(self, x, y):
        return (y-(5.1/(4*pi**2))*x**2+5*x/pi-6)**2+10*(1-1/(8*pi))*np.cos(x)+10
bh = BraninHoo()

In [ ]:
def plot_surface():
    x = np.linspace(bh.bounds[0][1], bh.bounds[0][2], 100)
    y = np.linspace(bh.bounds[1][1], bh.bounds[1][2], 100)

    X, Y = np.meshgrid(x, y)
    
    Z = bh.vectorized(X, Y)
    tp.surface_3D(X, Y, Z)
    return Z
Z = plot_surface()

In [ ]:
op = tb.Optimiser(bh, 'min', bh.bounds, pre_phase_trials=4, settings_preset='default')
op.acquisition = tm.UCB(beta=1)

op.surrogate = tm.SciKitGPSurrogate(model_params=dict(
    kernel = 1.0 * sk_gp.kernels.Matern(nu=2.5) + sk_gp.kernels.WhiteKernel(),
    normalize_y = True,
), training_iterations=5)


rec = tb.Recorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=15)

In [ ]:
tp.plot_overview(rec)

In [ ]:
tp.plot_error(rec, true_best=bh.optimal_y, log_scale=True);

In [ ]:
tp.interactive_plot_trial_1D(rec);

In [ ]:
tp.interactive_plot_trial_2D(rec, true_objective=Z);

# Try optimising the same function with random search

In [ ]:
ra = tb.Optimiser(bh, 'min', bh.bounds, pre_phase_trials=1, settings_preset='random_search')
recr = tb.Recorder(ra)
ra.run(max_trials=1000)

In [ ]:
tp.plot_error(recr, true_best=bh.optimal_y, log_scale=True);